In [1]:
import sqlite3
import pandas as pd
from datetime import datetime

In [13]:
from importlib import reload

In [47]:
import functions; reload(functions)
from functions import *

In [2]:
conn = sqlite3.connect('main.db')
c = conn.cursor()

In [4]:
datestring = '2024-01-01'

In [3]:
def get_stocks(from_date, to_date, max_n=5000):
    

In [29]:
ns = pd.read_sql_query('SELECT n FROM stocks', conn)

In [44]:
def get_max_value(var, database='main.db', table='stocks'):
    conn = sqlite3.connect(database)
    c = conn.cursor()
    max = c.execute(f'SELECT MAX({var}) FROM {table};').fetchone()[0]
    return max

In [34]:
c.execute(maxtime).fetchone()[0]

1718395200000

In [30]:
maxtime = 'SELECT MAX(time) FROM stocks;'

In [ ]:
abc = 

In [10]:
timestamp_from_datestring(datestring)

1704063600000

In [ ]:
datefrom

In [ ]:
onlylast = '''
    SELECT ticker FROM stocks
	WHERE time = (
		SELECT MAX(time) FROM stocks
	)
	ORDER BY volume * vw DESC
    LIMIT 1000;
'''

In [27]:
query = f'''
    SELECT * 
    FROM stocks 
    WHERE time > {timestamp_from_datestring('2024-01-01')}
    AND n > 1000
    '''

In [ ]:
def get_data(query):
    c.execute(query)
    data = c.fetchall()
    return data

In [22]:
df = pd.read_sql_query(query, conn)

In [23]:
df.shape

(521220, 9)

In [25]:
len(df.ticker.unique())

7101

In [26]:
df.shape

(521220, 9)

In [11]:
df.head()

,ticker,volume,vw,open,close,high,low,time,n
0,BTR,0.0,NaN,23.0399,23.0399,23.0399,23.0399,1704229200000,NaN
1,IFED,0.0,NaN,32.7126,32.7126,32.7126,32.7126,1704229200000,NaN
2,DKRB,0.0,NaN,22.9913,22.9913,22.9913,22.9913,1704229200000,NaN
3,AMND,0.0,NaN,40.8443,40.8443,40.8443,40.8443,1704229200000,NaN
4,FUSI,0.0,NaN,50.5550,50.5550,50.5550,50.5550,1704229200000,NaN


In [18]:
date_from_timestamp(df.time.min())

'2024-01-02'

In [48]:
timestamp_from_datestring('2024-01-01')

1704063600000